In [1]:
from orthofinder_utils import dash_app_preprocess as dap
from orthofinder_utils import proteomes_for_orthofinder as pfo
from orthofinder_utils import dash_ortho_parser as dop
from jw_utils import parse_gff as pgf
import os
import pandas as pd
import json
from Bio import Phylo
from jw_utils import ncbi_datasets_fxs as nfx
from jw_utils import jw_ncbi_taxonomy as jnt
from ete3 import ncbi_taxonomy
import shutil
ncbi_tax = ncbi_taxonomy.NCBITaxa()

ncbi.datasets module not found. To install, run `pip install ncbi-datasets-pylib`.


In [9]:
accessions = [f.strip('.faa') for f in os.listdir('./data/Proteomes/') if f.endswith('.faa')]

with open('accessions.txt', 'w') as f:
    for ele in accessions:
        f.write(ele + '\n')

In [11]:
!datasets summary genome accession --inputfile ./accessions.txt > ./data/summary_data/summaries.json

New version of client (15.12.0) available at https://ftp.ncbi.nlm.nih.gov/pub/datasets/command-line/LATEST/mac/datasets


In [14]:
dop_obj = dop.DashOrthoParser('./data')
self = dop_obj

FileNotFoundError: [Errno 2] No such file or directory: './data/summary_data/AssemblyAccession_to_SpeciesName.json'

### Setup for starting a orthofinder dash app    
***This is assuming you have run orthofinder on a set of proteomes and have the resulting    orthofinder data in a folder called Proteomes  
1. CHANGE WORKING DIRECTORY of this notebook TO dash_app main folder   
2. make a directory inside the main /dash_app folder called data (dash_app/data).
    - `$ !mkdir ./data`
    - `$ !mkdir ./data/Proteomes`
    - `$ !mv ./Proteomes/* ./data/Proteomes`

3. Place the ncbi_dataset/data containing all the data downloaded from ncbi into ./data
    - dash_app/data/ncbi_datasets/data  
4. You will need to generate a summary file of all genomes. Using the terminal is the most   
reliable way I've found to do this.  
    - `$ !datasets summary genome accession --inputfile accessions.txt > summaries.json`
 
 4) Run `dap.run_all()` in the cell below. If you haven't generated a summary file  
 then an error will be thrown, and code to get summary file via ncbi datasets cli will be  printed out


In [ ]:
pfo.grab_proteomes('./ncbi_dataset/data/')

### Check that species in different directories are equivalent

In [2]:
def check_for_equivalent_species():
    if not os.path.exists('./data/summary_data/'):
        os.makedirs('./data/summary_data/')
    if not os.path.exists('./accessions.txt'):
        raise FileNotFoundError('"./accessions.txt" could not be found.')
    proteomes = [f[:15] for f in os.listdir('./data/Proteomes') if f.endswith('.faa')]
    assemblies = [f[:15] for f in os.listdir('./data/ncbi_dataset/data') if f.startswith('GC')]
    with open('./accessions.txt', 'r') as f:
        accessions = [line.strip() for line in f]
    if not sorted(accessions) == sorted(assemblies):
        raise Exception(f'"./data/ncbi_dataset/data" and "./accessions.txt" are not equivalent') 
    else:
        print(f'"./data/ncbi_dataset/data" and "./accessions.txt" have the same genome assemblies') 
    if not sorted(assemblies) == sorted(proteomes):
        raise Exception(f'"./data/ncbi_dataset/data" and "./data/Proteomes" are not equivalent')
    else:
        print(f'"./data/ncbi_dataset/data" and "./data/Proteomes" have the same genome assemblies/proteomes') 
    if not sorted(accessions) == sorted(proteomes):
        raise Exception(f'"./accessions.txt" and "./data/Proteomes" are not equivalent')
    else:
        print(f'"./accessions.txt" and "./data/Proteomes" have the same genome assemblies/proteomes') 
    shutil.copy('./accessions.txt', './data/summary_data/accessions.txt')
    
accs_proteomes = [a[:15] for a in os.listdir('./data/Proteomes/') if a.startswith('GC')]
acc_assemblies = [f[:15] for f in os.listdir('./data/ncbi_dataset/data') if f.startswith('GC')]

def deduplicate_accessions_file():
    with open('./accessions.txt', 'r') as f:
        accs_from_file = []
        for line in f: 
            accs_from_file.append(line.strip())
    with open('./accessions.txt', 'w') as f:
        accs_from_file = list(set(accs_from_file))
        for acc in accs_from_file:
            f.write(f'{acc}\n')
    return accs_from_file

accs_from_file = deduplicate_accessions_file()
accs_from_file.sort()
acc_assemblies.sort()
accs_proteomes.sort()
acc_assemblies == accs_proteomes
accs_from_file == accs_proteomes

for acc in accs_proteomes:
    if acc not in accs_from_file:
        print(f'Proteome {acc} not in ./accessions.txt')
for acc in accs_from_file:
    if acc not in accs_proteomes:
        print(f'Assession {acc} from ./assessions.txt  not in ./data/Proteomes/')
for acc in acc_assemblies:
    if acc not in accs_proteomes:
        print(f'Assession {acc} from ./data/ncbi_dataset/data  not in ./data/Proteomes/')
        print(acc)
        
check_for_equivalent_species()

"./data/ncbi_dataset/data" and "./accessions.txt" have the same genome assemblies
"./data/ncbi_dataset/data" and "./data/Proteomes" have the same genome assemblies/proteomes
"./accessions.txt" and "./data/Proteomes" have the same genome assemblies/proteomes


In [17]:
os.getcwd()

'/Users/jonwinkelman/Trestle_projects/Mukherjee_lab/dash_app_pseudomonas'

In [33]:
!datasets summary genome accession --inputfile ./data/summary_data/accessions.txt > ./data/summary_data/summaries.json

New version of client (15.11.0) available at https://ftp.ncbi.nlm.nih.gov/pub/datasets/command-line/LATEST/mac/datasets


## Generate data files for dash app
### !!Change path for latest orthofinder results!!

In [3]:
path_to_parent=os.getcwd()
path_to_results = './data/Proteomes/OrthoFinder/Results_Aug04_1'
dap.run_all(path_to_results)

FileNotFoundError: [Errno 2] No such file or directory: './data/Proteomes/OrthoFinder/Results_Aug04_1/Orthogroups'

In [4]:
def make_internal_node_d(tree):
    """"""
    internal_node_dict = {}
    for node in tree.get_nonterminals():
        internal_node_dict[node.name] = {'name':node.name, 'rank':'', 'taxid':'', 'sci_name':''}
    for node in tree.get_terminals():
            internal_node_dict[node.name] = {'name':node.name}
    internal_node_dict

    with open('./data/summary_data/internal_node_dict.json', 'w') as f:
        json.dump(internal_node_dict, f)
    return internal_node_dict

def accession_taxid_d(path_to_summary):
    """Return simplified dict {acc:taxid} from ncbi summary.json."""
    
    with open(path_to_summary, 'r') as f:
        summary_d = nfx.make_summary_dict(json.load(f))
    acc_tax_d ={}
    for acc in summary_d:
        acc_tax_d[acc] = summary_d[acc]['org']['tax_id']
    return acc_tax_d


def get_rank_sciname_for_each_leaf(tree, rank, path_to_summary):
    """Return a dict with a sci_name at a given rank for each leaf {leaf:sci_name}
    e.g. given rank phylum: return {GCF_######:'Proteobacteria'}
    """

    acc_tax_d =  accession_taxid_d(path_to_summary)   
    leaf_rank_sciname = {}
    for leaf in tree.get_terminals():
        leaf_taxid = acc_tax_d[leaf.name]
        leaf_rank_taxid = jnt.get_lineage_rank_dict(leaf_taxid).get(rank)
        if leaf_rank_taxid:
            leaf_rank_sciname[leaf.name] = list(ncbi_tax.get_taxid_translator([leaf_rank_taxid]).values())[0]
        else:
            leaf_rank_sciname[leaf.name] = None
    return leaf_rank_sciname


def get_list_of_all_nodes(subtree, clades=None):
    """Get a list of all clade objects, including leaves, in a Bio.Phylo tree"""
    if not isinstance(subtree, (Phylo.Newick.Tree, Phylo.Newick.Clade)):
        raise TypeError(f'object entered needs to be of type {Phylo.Newick.Tree} or {Phylo.Newick.Clade}, you entered {type(subtree)}' )
        
    if clades is None:
        clades=[]
    for cl in subtree.root:
        clades.append(cl)
        if cl.is_terminal():
             clades.append(cl)
        else:
            get_list_of_all_nodes(cl, clades)
    return clades


def get_nodes_assoc_with_ranks(tree, rank, path_to_summary):
    """"""
    leaf_rank_scinames_d = get_rank_sciname_for_each_leaf(tree, rank, path_to_summary)
    node_dict = {}
    for cl in get_list_of_all_nodes(tree):
        #check to see if all leaves in clade have same sci_name, if so then clade is that sci_name
        leaves =  cl.get_terminals()
        sci_names = []

        for leaf in leaves:
            sci_names.append(leaf_rank_scinames_d[leaf.name])
        sci_names = list(set(sci_names))
        if len(sci_names) == 1:
            node_dict[cl.name] = sci_names[0]
    unique_scinames = set(list(node_dict.values()))
    rank_nodes = {n:[] for n in unique_scinames}
    for node, sci_name in node_dict.items():
        rank_nodes[sci_name].append(node)
    return rank_nodes


def get_anc_node_each_rank(tree, rank, path_to_summary):
    """"""
    rank_nodes = get_nodes_assoc_with_ranks(tree, rank, path_to_summary)
    mrca_clades = {}
    for rank_name, clades in rank_nodes.items():
        leaves = []
        for cl in clades:
            clade = tree.find_any(cl) 
            if clade.is_terminal():
                leaves.append(clade)
        mrca_clade  = tree.is_monophyletic(leaves)
        if mrca_clade:
            mrca_clades[rank_name] = mrca_clade.name
        else:
            mrca_clades[rank_name] = 'not monophyletic'
    return mrca_clades


def make_mrcaClade_file(tree, path_to_summary):
    rank_mrca_clades_d = {}
    available_ranks = ['no rank', 'superkingdom','family', 'genus', 'phylum', 'class', 'order', 'species']
    for rank in available_ranks:
        mrca_clades = get_anc_node_each_rank(tree, rank, path_to_summary)
        rank_mrca_clades_d[rank] = mrca_clades
    with open('./ranks_mrca_clades.json', 'w') as f:
        json.dump(rank_mrca_clades_d, f)
    return rank_mrca_clades_d  



tree = Phylo.read('./data/Species_Tree/SpeciesTree_rooted_node_labels.txt', format='newick')
internal_node_dict = make_internal_node_d(tree)
path_to_summary ='./data/summary_data/summaries.json'
rank_mrca_clades_d = make_mrcaClade_file(tree, path_to_summary)


### Done

In [5]:
ass2_name_path = './data/summary_data/AssemblyAccession_to_SpeciesName.json'

In [6]:
def make_ass2_name_dict(ass2_name_path):
    """Return accession_to_name dict from AssemblyAccession_to_SpeciesName.json in summary data""" 
    with open(ass2_name_path, 'r') as f:
        return json.load(f)

In [8]:
make_ass2_name_dict(ass2_name_path)

{'GCF_024448735.1': '[Pseudomonas] zhaodongensis DSM 27559T',
 'GCA_001077675.1': 'Acinetobacter baumannii ATCC 17978-mff',
 'GCF_000046845.1': 'Acinetobacter baylyi ADP1',
 'GCF_000413935.1': 'Acinetobacter colistiniresistens NIPH 2036',
 'GCF_001682515.1': 'Acinetobacter gyllenbergii FMP01',
 'GCF_005281455.1': 'Acinetobacter nosocomialis M2',
 'GCF_002362295.1': 'Burkholderia ubonensis subsp. mesacidophila',
 'GCF_004124255.1': 'Candidatus Pseudomonas adelgestsugas',
 'GCF_900105005.1': 'Halopseudomonas litoralis 2SM5',
 'GCF_020025155.1': 'Halopseudomonas nanhaiensis SCS 2-3',
 'GCF_009497895.1': 'Halopseudomonas pelagia Kongs-67',
 'GCF_900105255.1': 'Halopseudomonas sabulinigri JCM 14963',
 'GCF_900105655.1': 'Halopseudomonas salegens CECT 8338',
 'GCF_900104945.1': 'Halopseudomonas xinjiangensis NRRL B-51270',
 'GCF_004355125.1': 'Pseudomonas aeruginosa AES1M',
 'GCF_004355145.1': 'Pseudomonas aeruginosa AES1R',
 'GCF_011466835.1': 'Pseudomonas aeruginosa CF39S',
 'GCF_013343435